In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader,random_split
from torchvision import transforms, datasets, models
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn.utils.prune as prune

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean = [0.4914, 0.4822, 0.4465] 
std = [0.2470, 0.2435, 0.2616] 
batch_size = 40
n_epochs = 100

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    # Random augmentations
    # Randomly rotate images by 40 degrees
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05),  # Random color jitter
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=mean, std=std)  # Normalize with mean and std
])

path='train'
all_train = datasets.ImageFolder(root = path, transform = train_transform)
test = datasets.ImageFolder(root = 'ttest', transform = train_transform)
train_size = int(0.9 * len(all_train))
validation_size = len(all_train) - train_size
train_dataset, validation_dataset = random_split(all_train , [train_size, validation_size])
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)
val_loader = DataLoader(
    validation_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)
test_loader = DataLoader(
    validation_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)

In [3]:
image_size = (3, 224, 224)  # Example: 3 channels, 32x32 pixels
num_classes = 100

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import ceil

# Inverted Residual Block with Squeeze-and-Excitation
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expand_ratio, stride, kernel_size, reduction_ratio=4):
        super(MBConvBlock, self).__init__()
        self.stride = stride
        self.expand_ratio = expand_ratio
        hidden_dim = in_channels * expand_ratio
        
        # Expansion phase
        if expand_ratio != 1:
            self.expand_conv = nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False)
            self.bn0 = nn.BatchNorm2d(hidden_dim)
        else:
            self.expand_conv = None
        
        # Depthwise convolution
        self.depthwise_conv = nn.Conv2d(hidden_dim if expand_ratio != 1 else in_channels, hidden_dim, 
                                        kernel_size=kernel_size, stride=stride, 
                                        padding=kernel_size // 2, groups=hidden_dim, bias=False)
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        
        # Squeeze and Excitation block
        self.se_avgpool = nn.AdaptiveAvgPool2d(1)
        self.se_fc1 = nn.Conv2d(hidden_dim, hidden_dim // reduction_ratio, kernel_size=1)
        self.se_fc2 = nn.Conv2d(hidden_dim // reduction_ratio, hidden_dim, kernel_size=1)
        
        # Output phase
        self.project_conv = nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.use_residual = (in_channels == out_channels and stride == 1)
    
    def forward(self, x):
        identity = x
        
        if self.expand_conv:
            out = F.relu6(self.bn0(self.expand_conv(x)))
        else:
            out = x
        
        # Depthwise convolution
        out = F.relu6(self.bn1(self.depthwise_conv(out)))
        
        # Squeeze and Excitation
        se = self.se_avgpool(out)
        se = F.relu(self.se_fc1(se))
        se = torch.sigmoid(self.se_fc2(se))
        out = out * se
        
        # Output
        out = self.bn2(self.project_conv(out))
        
        if self.use_residual:
            out = out + identity
        
        return out

# EfficientNet Main Architecture
class EfficientNet(nn.Module):
    def __init__(self, width_coefficient=0.6, depth_coefficient=0.7, dropout_rate=0.2, num_classes=100):
        super(EfficientNet, self).__init__()
        
        # Base settings for EfficientNet with reduced coefficients
        base_channels = 16  # Further reduced base channels
        base_layers = [
            # (expand_ratio, out_channels, num_blocks, stride, kernel_size)
            (1, 16, 1, 1, 3),   # Stage 1
            (6, 24, 1, 2, 3),   # Stage 2
            (6, 40, 1, 2, 5),   # Stage 3
            (6, 80, 2, 2, 3),   # Stage 4
            (6, 112, 2, 1, 5),  # Stage 5
            (6, 192, 2, 2, 5),  # Stage 6
            (6, 320, 1, 1, 3)   # Stage 7
        ]
        
        # Stem
        out_channels = ceil(base_channels * width_coefficient)
        self.stem_conv = nn.Conv2d(3, out_channels, kernel_size=3, stride=2, padding=1, bias=False)
        self.stem_bn = nn.BatchNorm2d(out_channels)
        
        # Build blocks
        self.blocks = nn.ModuleList([])
        in_channels = out_channels
        for expand_ratio, out_channels, num_blocks, stride, kernel_size in base_layers:
            out_channels = ceil(out_channels * width_coefficient)
            num_blocks = ceil(num_blocks * depth_coefficient)
            for i in range(num_blocks):
                block_stride = stride if i == 0 else 1
                self.blocks.append(MBConvBlock(in_channels, out_channels, expand_ratio, block_stride, kernel_size))
                in_channels = out_channels
        
        # Head
        final_channels = ceil(640 * width_coefficient)  # Further reduced head channels
        self.head_conv = nn.Conv2d(in_channels, final_channels, kernel_size=1, bias=False)
        self.head_bn = nn.BatchNorm2d(final_channels)
        
        # Pooling and classification
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(final_channels, num_classes)
    
    def forward(self, x):
        # Stem
        x = F.relu6(self.stem_bn(self.stem_conv(x)))
        
        # Blocks
        for block in self.blocks:
            x = block(x)
        
        # Head
        x = F.relu6(self.head_bn(self.head_conv(x)))
        
        # Pooling and classification
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

def efficientnet_custom(num_classes=100):
    return EfficientNet(width_coefficient=0.6, depth_coefficient=0.7, num_classes=num_classes)

In [13]:
model =EfficientNet().to(device)
prune.l1_unstructured(model.fc, name='weight', amount=0.4)
total_params = sum(p.numel() for p in model.parameters())
print("# parameters:", total_params)

# parameters: 1521845


In [14]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)

# Early stopping class
class EarlyStopper:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

# Train function
def train(model, train_loader, optimizer, loss_fn):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy

# Validation function
@torch.no_grad()
def validate(model, test_loader, loss_fn):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        val_loss += loss.item() * images.size(0)
        
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
    avg_loss = val_loss / len(test_loader.dataset)
    accuracy = correct / total
    return avg_loss, accuracy

# Training loop
train_loss_list = []
valid_loss_list = []
early_stopper = EarlyStopper(patience=7)

best_val_acc = 0.0

for epoch in range(n_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn)
    val_loss, val_acc = validate(model, val_loader, loss_fn)
    
    train_loss_list.append(train_loss)
    valid_loss_list.append(val_loss)
    
    print(f"Epoch {epoch+1}/{n_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Step the learning rate scheduler
    scheduler.step()
    
    # Early stopping check
    if early_stopper.early_stop(val_loss):
        print("Early stopping")
        break
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

# Save the final model
torch.save(model.state_dict(), "final_model.pth")

100%|██████████| 35/35 [00:17<00:00,  2.04it/s]


Epoch 1/100:
Train Loss: 4.0365, Train Acc: 0.0667
Val Loss: 3.7065, Val Acc: 0.1000


100%|██████████| 35/35 [00:11<00:00,  3.14it/s]


Epoch 2/100:
Train Loss: 3.4492, Train Acc: 0.1374
Val Loss: 3.4044, Val Acc: 0.1471


100%|██████████| 35/35 [00:13<00:00,  2.67it/s]


...

Epoch 52/100:
Train Loss: 0.3852, Train Acc: 0.8808
Val Loss: 1.1514, Val Acc: 0.7171


100%|██████████| 35/35 [00:09<00:00,  3.75it/s]


Epoch 53/100:
Train Loss: 0.3606, Train Acc: 0.8877
Val Loss: 1.1308, Val Acc: 0.7229


100%|██████████| 35/35 [00:09<00:00,  3.79it/s]

Epoch 54/100:
Train Loss: 0.3392, Train Acc: 0.8960
Val Loss: 1.1217, Val Acc: 0.7250
Early stopping


In [19]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os
import pandas as pd

# Custom Dataset for Test Images
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
# Import the custom EfficientNet model
# prune.remove(model.fc, 'weight')

# # Then save the model state
# torch.save(model.state_dict(), 'best_model.pth')

def test():
    # Load the trained model
    model = EfficientNet()  # Initialize the model with 100 classes
    model.load_state_dict(torch.load('best_model.pth'))  # Assuming you saved the trained model as model.pth
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # # Data transformations
    # transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    # ])

    # Load the test dataset
    # test_dataset = datasets.ImageFolder(root='ttest', transform=transform)
    # test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
    print(f'Test Accuracy: {accuracy}')

if __name__ == "__main__":
    test()
#----------------------------------------------------------------------------------------------------
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.file_names = os.listdir(test_dir)
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_name = self.file_names[idx]
        img_path = os.path.join(self.test_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = efficientnet_custom(num_classes=100).to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Define transformations for train and test datasets
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    # Random augmentations
    # Randomly rotate images by 40 degrees
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05),  # Random color jitter
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=mean, std=std)  # Normalize with mean and std
])

# Load the train dataset using ImageFolder
train_dir = os.path.abspath('./train')  # Replace with the path to your train dataset
train_dataset = ImageFolder(train_dir, transform=transform)

# Get the class_to_idx from the training dataset and invert it to create idx_to_class
class_to_idx = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}  # Invert class_to_idx to idx_to_class

# Load test dataset
test_dir = os.path.abspath('./test')  # Path to your test dataset
test_dataset = TestDataset(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Prepare CSV for results
results = []

# Make predictions and store them in result.csv
with torch.no_grad():
    for images, file_names in test_loader:  # file_names returned from the dataset
        images = images.to(device)  # Move images to GPU/CPU
        outputs = model(images)
        
        # Get top-5 predictions for each image
        _, top5_preds = torch.topk(outputs, 5, dim=1)
        
        for i, preds in enumerate(top5_preds):
            file_name = file_names[i]  # Get the file name
            # Convert predicted indices to class labels (words)
            pred_labels = [idx_to_class[pred.item()] for pred in preds]
            
            # Append to results
            results.append([file_name] + pred_labels)

# Save results to CSV
df = pd.DataFrame(results, columns=['file_name', 'pred1', 'pred2', 'pred3', 'pred4', 'pred5'])
df.to_csv('result.csv', index=False)

print("Predictions saved to result.csv")
#---------------------------------------------------------------------------------------------------------



# def test():
#     # Load the trained model
#     model = efficientnet_custom(num_classes=100) 
#     model.load_state_dict(torch.load('best_model.pth'))  # Load the trained model
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = model.to(device)

#     # Data transformations
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])

#     # Load the test dataset
#     test_dataset = datasets.ImageFolder(root='ttest', transform=transform)
#     test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
#      # Mapping from index to class label (word)
#     idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}

#     # Create CSV file with predictions
#     with open('result.csv', 'w') as f:
#         # Write header
#         f.write('file_name,pred1,pred2,pred3,pred4,pred5\n')

#         # Make predictions and store them in result.csv
#         with torch.no_grad():
#             for i, (images, labels) in enumerate(test_loader):
#                 outputs = model(images)
#                 _, top5_preds = torch.topk(outputs, 5, dim=1)

#                 for j, preds in enumerate(top5_preds):
#                     # Get the file name for the current image
#                     file_name = test_dataset.samples[i * test_loader.batch_size + j][0]

#                     # Convert predicted indices to class labels (words)
#                     pred_labels = [idx_to_class[pred.item()] for pred in preds]

#                     # Write to CSV
#                     f.write(f'{file_name},{pred_labels[0]},{pred_labels[1]},{pred_labels[2]},{pred_labels[3]},{pred_labels[4]}\n')

# if __name__ == "__main__":
#     test()

#     results = []

#     with torch.no_grad():
#         for images, labels in test_loader:
#             images = images.to(device)
#             outputs = model(images)
#             # Get the top 5 predictions
#             _, top5 = torch.topk(outputs, 5, dim=1)

#             # Loop through the batch and save results
#             for i, idx in enumerate(top5):
#                 file_name = test_dataset.samples[i][0]  # Get the image file path
#                 file_name = os.path.basename(file_name)  # Get the file name only
#                 pred_labels = idx.cpu().numpy().tolist()  # Convert to list of top 5 predictions
#                 results.append([file_name] + pred_labels)

#     # Write the results to a CSV file
#     with open('result.csv', mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['file_name', 'pred1', 'pred2', 'pred3', 'pred4', 'pred5'])  # Write header
#         writer.writerows(results)  # Write all rows of results

#     print(f'Results saved to result.csv')

# if __name__ == "__main__":
#     test()


Test Accuracy: 93.93%
Test Accuracy: 0.9392857142857143
Predictions saved to result.csv
